In [1]:
!pip install pyabsa scikit-learn wandb -U
!pip install textattack

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 574.2/574.2 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 116.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 101.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 14.8 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=d53a4e575dc0263b4c76bc9e162bf0cf56fb15bf172a0c297171f8887f5dad48
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
  Attempting uninst

In [2]:
!pip show transformers

Name: transformers
Version: 4.57.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.12/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: bert-score, flair, kaggle-environments, peft, pyabsa, sentence-transformers, textattack, transformer-smaller-training-vocab


# Data Glance

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import ast

In [4]:
data = pd.read_csv('/kaggle/input/d/namdtgk14/aspect-based-sentiment-analysis-for-financial-news/SEntFiN-v1.1_with_split.csv')
data.head(10)

,S No.,Title,Decisions,Words,split
0,1,SpiceJet to issue 6.4 crore warrants to promoters,"{""SpiceJet"": ""neutral""}",8,test
1,2,MMTC Q2 net loss at Rs 10.4 crore,"{""MMTC"": ""neutral""}",8,train
2,3,"Mid-cap funds can deliver more, stay put: Experts","{""Mid-cap funds"": ""positive""}",8,train
3,4,Mid caps now turn into market darlings,"{""Mid caps"": ""positive""}",7,train
4,5,"Market seeing patience, if not conviction: Pra...","{""Market"": ""neutral""}",8,train
5,6,Infosys: Will the strong volume growth sustain?,"{""Infosys"": ""neutral""}",7,train
6,7,Hudco raises Rs 279 cr via tax-free bonds,"{""Hudco"": ""positive""}",8,train
7,8,HOEC could retest 30-35 levels: Ashwani Gujral,"{""HOEC"": ""neutral""}",7,train
8,9,Gold shines on seasonal demand; Silver dull,"{""Gold"": ""positive"", ""Silver"": ""negative""}",7,train
9,10,Genpact appoints Edward J Fitzpatrick as CFO,"{""Genpact"": ""neutral""}",7,train


In [5]:
import pandas as pd

# Assuming your DataFrame is named 'df'

# 1. Split the data based on the 'split' column values
train_data = data[data['split'] == 'train'].copy()
test_data = data[data['split'] == 'test'].copy()
val_data = data[data['split'] == 'val'].copy() # Assumes the value is named 'val'

# 2. Remove the 'split' column from each set
train_data = train_data.drop(columns=['split'])
test_data = test_data.drop(columns=['split'])
val_data = val_data.drop(columns=['split'])

# Optional: Reset index if you want them to start from 0
train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)
val_data = val_data.reset_index(drop=True)

# Verification
print(f"Train shape: {train_data.shape}")
print(f"Test shape: {test_data.shape}")
print(f"Val shape: {val_data.shape}")

Train shape: (7693, 4)
Test shape: (2227, 4)
Val shape: (833, 4)


In [6]:
# Len of data point in train and test dataset

print("train data: ", len(train_data))
print("validation data: ", len(val_data))
print("test data: ", len(test_data))

train data:  7693
validation data:  833
test data:  2227


In [7]:
train_data.head(5)

,S No.,Title,Decisions,Words
0,2,MMTC Q2 net loss at Rs 10.4 crore,"{""MMTC"": ""neutral""}",8
1,3,"Mid-cap funds can deliver more, stay put: Experts","{""Mid-cap funds"": ""positive""}",8
2,4,Mid caps now turn into market darlings,"{""Mid caps"": ""positive""}",7
3,5,"Market seeing patience, if not conviction: Pra...","{""Market"": ""neutral""}",8
4,6,Infosys: Will the strong volume growth sustain?,"{""Infosys"": ""neutral""}",7


# PyABSA




In [8]:
from pyabsa import AspectPolarityClassification as APC

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
2025-12-21 14:30:37.133113: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766327437.319388      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766327437.375230      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766327437.826382      24 computation_placer.cc:177] computation placer already registered. Please check

[2025-12-21 14:30:54] (2.4.2) PyABSA(2.4.2): If your code crashes on Colab, please use the GPU runtime. Then run "pip install pyabsa[dev] -U" and restart the kernel.
Or if it does not work, you can use v1.x versions, e.g., pip install pyabsa<2.0 -U




Try to downgrade transformers<=4.29.0.






## APCDataset

In [9]:
import os
import shutil
from pyabsa.framework.flag_class.flag_template import LabelPaddingOption

def process_dataframe(df, output_path, task='apc', txt_col='Title', label_col='Decisions'):
    """
    Helper function to process a single DataFrame and write to the output file
    based on the selected task.
    """
    
    # Normalize column names
    df.columns = df.columns.str.strip()
    
    # Check columns
    if txt_col not in df.columns or label_col not in df.columns:
        # Fallback for alternative headers found in some Kaggle versions
        if 'News Headlines' in df.columns and 'Sentiments' in df.columns:
            txt_col, label_col = 'News Headlines', 'Sentiments'
        else:
            print(f"  [Error] Missing columns in data. Needed '{txt_col}' & '{label_col}'. Found: {df.columns}")
            return

    with open(output_path, 'w', encoding='utf-8') as f_out:
        count = 0
        for _, row in df.iterrows():
            text = str(row[txt_col]).strip()
            
            # Parse the dictionary string: '{"Entity": "Sentiment"}'
            try:
                # If the data is already a dict (not string), use it directly
                if isinstance(row[label_col], dict):
                    decisions = row[label_col]
                else:
                    decisions = ast.literal_eval(row[label_col])
            except (ValueError, SyntaxError):
                continue

            if not decisions:
                continue

            # --- APC GENERATION ---
            if task == 'apc':
                for aspect, sentiment in decisions.items():
                    if aspect not in text:
                        continue
                    
                    # Replace aspect with $T$
                    text_apc = text.replace(aspect, "$T$", 1)
                    
                    f_out.write(f"{text_apc}\n")
                    f_out.write(f"{aspect}\n")
                    f_out.write(f"{sentiment}\n")
                    count += 1

            # --- ATEPC GENERATION ---
            elif task == 'atepc':
                tokens = text.split()
                bio_labels = ['O'] * len(tokens)
                sentiment_labels = [str(LabelPaddingOption.LABEL_PADDING)] * len(tokens)
                
                found_aspect = False
                for aspect, sentiment in decisions.items():
                    aspect_tokens = aspect.split()
                    
                    # Simple exact match finder
                    for i in range(len(tokens) - len(aspect_tokens) + 1):
                        if tokens[i : i + len(aspect_tokens)] == aspect_tokens:
                            bio_labels[i] = "B-ASP"
                            sentiment_labels[i] = sentiment
                            for j in range(1, len(aspect_tokens)):
                                bio_labels[i + j] = "I-ASP"
                                sentiment_labels[i + j] = sentiment
                            found_aspect = True
                            break 
                
                # Only write if we actually found/labeled an aspect
                if found_aspect:
                    for token, bio, sent in zip(tokens, bio_labels, sentiment_labels):
                        f_out.write(f"{token} {bio} {sent}\n")
                    f_out.write("\n")
                    count += 1

    print(f"  -> Wrote {count} samples to {os.path.basename(output_path)}")


def create_pyabsa_dataset(
    dataset_name, 
    train_df, 
    test_df, 
    val_df=None, 
    task='apc', 
    root_output_dir="pyabsa_custom_dataset"
):
    """
    Main function to orchestrate the dataset creation using DataFrames.
    
    Args:
        dataset_name (str): Name of your dataset (e.g., 'FinancialNews').
        train_df (pd.DataFrame): Training DataFrame.
        test_df (pd.DataFrame): Testing DataFrame.
        val_df (pd.DataFrame): Validation DataFrame (Optional).
        task (str): 'apc', 'atepc', or 'both'.
        root_output_dir (str): Where to create the folders.
    """
    
    tasks_to_process = ['apc', 'atepc'] if task == 'both' else [task]
    
    for current_task in tasks_to_process:
        # 1. Create Directory Structure
        # Structure: root_output_dir / dataset_name / 
        target_dir = os.path.join(root_output_dir, f"{dataset_name}_{current_task}")
        os.makedirs(target_dir, exist_ok=True)
        
        print(f"\n=== Generating {current_task.upper()} Dataset at: {target_dir} ===")

        # 2. Define DataFrame Mapping
        dfs_map = {
            'train': train_df,
            'test': test_df
        }
        if val_df is not None and not val_df.empty:
            dfs_map['valid'] = val_df
            
        # 3. Process Each Split
        for split_name, df in dfs_map.items():
            print(f"Processing {split_name} set...")
            
            if df is None or df.empty:
                print(f"  [Warning] {split_name} DataFrame is empty. Skipping.")
                continue

            # PyABSA standard naming: dataset_name.train.dat.apc
            output_filename = f"{split_name}.dat.{current_task}"
            output_path = os.path.join(target_dir, output_filename)
            
            process_dataframe(df, output_path, task=current_task)

In [10]:
create_pyabsa_dataset(
        dataset_name="SEntFiN",
        train_df=train_data,   # Pass your actual train DataFrame here
        test_df=test_data,     # Pass your actual test DataFrame here
        val_df=val_data,            # Pass your actual val DataFrame here
        task='apc',              # Choose 'apc', 'atepc', or 'both'
        root_output_dir="/kaggle/working/"
    )


=== Generating APC Dataset at: /kaggle/working/SEntFiN_apc ===
Processing train set...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


  -> Wrote 10284 samples to train.dat.apc
Processing test set...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


  -> Wrote 3000 samples to test.dat.apc
Processing valid set...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


  -> Wrote 1125 samples to valid.dat.apc


In [11]:
# from pyabsa.utils.data_utils import validate_dataset

# # Validate your dataset format
# is_valid, errors = validate_dataset("/kaggle/working/SEntFiN", task="apc")

# if not is_valid:
#     print("Dataset validation errors:")
#     for error in errors:
#         print(f"- {error}")

In [12]:
from pyabsa import DatasetItem
dataset = DatasetItem("SEntFiN", ["/kaggle/working/SEntFiN_apc"])

## Augment

### bug fix 

In [13]:
# import os
# import shutil

# # 1. Define the paths
# # Your current data is here (generated by your previous function)
# generated_data_source = "/kaggle/working/SEntFiN" 

# # PyABSA is strictly looking for this specific folder structure:
# target_backup_dir = "source_datasets.backup"
# dataset_name = "SEntFiN"
# target_dataset_path = os.path.join(target_backup_dir, dataset_name)

# # 2. Create the "Trojan Horse" directory
# if not os.path.exists(target_dataset_path):
#     os.makedirs(target_dataset_path)
#     print(f"Created directory: {target_dataset_path}")

# # 3. Copy your .dat.apc files into that directory
# print(f"Copying files from {generated_data_source} to {target_dataset_path}...")

# # Check if source exists
# if os.path.exists(generated_data_source):
#     for filename in os.listdir(generated_data_source):
#         # We only want the data files
#         if filename.endswith(".dat.apc"):
#             src = os.path.join(generated_data_source, filename)
#             dst = os.path.join(target_dataset_path, filename)
#             shutil.copy2(src, dst)
#             print(f"  -> Copied {filename}")
# else:
#     print(f"Error: Could not find your generated data at {generated_data_source}")
    
# print("\nDone! PyABSA will now find 'SEntFiN' in the backup folder.")

In [14]:
# from pyabsa.augmentation import auto_aspect_sentiment_classification_augmentation
# import warnings

# warnings.filterwarnings('ignore')

# # Configure your model
# config_a = APC.APCConfigManager.get_apc_config_english()
# config_a.model = APC.APCModelList.FAST_LSA_T_V2
# config_a.pretrained_bert = 'microsoft/deberta-v3-base'
# config_a.num_epoch = 20
# config_a.evaluate_begin = 5
# config_a.max_seq_len = 80
# config_a.dropout = 0.1
# config_a.l2reg = 1e-8

In [15]:
# # Automatic augmentation and training
# auto_aspect_sentiment_classification_augmentation(
#     config=config_a,
#     dataset=dataset,
#     device='cuda',
#     augment_backend='EDA',
# )

## Config

In [16]:
from metric_visualizer import MetricVisualizer

# 1. Create a MetricVisualizer object
# mv = MetricVisualizer()

In [17]:
config = APC.APCConfigManager.get_apc_config_english()

In [18]:
config.model = APC.APCModelList.FAST_LSA_T_V2
# config.MV = mv

config.num_epoch = 25
config.patience = 3
config.warmup_step = 5
config.seed = 42
config.learning_rate = 1e-5
config.l2reg = 1e-3
config.lsa = True

## Train

In [19]:

import shutil
import time

import numpy
import numpy as np
import pandas
import torch
import torch.nn as nn
from sklearn import metrics
from torch import cuda
from tqdm import tqdm
import wandb  # <--- Added WandB import
from typing import cast

from pyabsa.framework.flag_class.flag_template import DeviceTypeOption
from pyabsa.framework.instructor_class.instructor_template import BaseTrainingInstructor
from pyabsa.tasks.AspectPolarityClassification.instructor.ensembler import APCEnsembler
from pyabsa.utils.file_utils.file_utils import save_model
from pyabsa.utils.pyabsa_utils import init_optimizer, fprint

In [20]:
wandb.login(key = "d9d14819dddd8a35a353b5c0b087e0f60d717140")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
/usr/local/lib/python3.12/dist-packages/urllib3/response.py:221: DependencyWarning: Brotli >= 1.2.0 is required to prevent decompression bombs.
  warnings.warn(
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


True

sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x78dd3479eac0>


In [21]:
class APCTrainingInstructorWandB(BaseTrainingInstructor):
    def _load_dataset_and_prepare_dataloader(self):
        self.model = APCEnsembler(self.config)
        self.tokenizer = self.model.tokenizer

        self.train_set = self.model.train_set
        self.test_set = self.model.test_set
        self.test_dataloader = self.model.test_dataloader
        self.valid_dataloader = self.model.valid_dataloader
        self.train_dataloader = self.model.train_dataloader

    def __init__(self, config):
        super().__init__(config)

        self._load_dataset_and_prepare_dataloader()
        self._init_misc()

        # --- WandB Initialization ---
        # Check if wandb is already initialized to avoid re-init errors
        if wandb.run is None:
            wandb.init(
                project=config.get("wandb_project", "PyABSA_APC"),
                entity=config.get("wandb_entity", None),
                name=f"{config.model_name}_{config.dataset_name}",
                config=config.args if hasattr(config, "args") else vars(config),
                reinit=True
            )
        # ----------------------------

    def _train_and_evaluate(self, criterion):
        global_step = 0
        max_fold_acc = 0
        max_fold_f1 = 0
        save_path = "{0}/{1}_{2}".format(
            self.config.model_path_to_save,
            self.config.model_name,
            self.config.dataset_name,
        )

        self.config.metrics_of_this_checkpoint = {"acc": 0, "f1": 0}
        self.config.max_test_metrics = {"max_apc_test_acc": 0, "max_apc_test_f1": 0}

        losses = []

        Total_params = 0
        Trainable_params = 0
        NonTrainable_params = 0

        for param in self.model.parameters():
            mulValue = numpy.prod(param.size())
            Total_params += mulValue
            if param.requires_grad:
                Trainable_params += mulValue
            else:
                NonTrainable_params += mulValue

        patience = self.config.patience + self.config.evaluate_begin
        if self.config.log_step < 0:
            self.config.log_step = (
                len(self.train_dataloaders[0])
                if self.config.log_step < 0
                else self.config.log_step
            )

        self.logger.info("***** Running training for {} *****".format(self.config.task_name))
        self.logger.info("Training set examples = %d", len(self.train_set))
        if self.valid_set:
            self.logger.info("Valid set examples = %d", len(self.valid_set))
        if self.test_set:
            self.logger.info("Test set examples = %d", len(self.test_set))
        self.logger.info(
            "Total params = %d, Trainable params = %d, Non-trainable params = %d",
            Total_params,
            Trainable_params,
            NonTrainable_params,
        )
        self.logger.info("Batch size = %d", self.config.batch_size)
        self.logger.info(
            "Num steps = %d",
            len(self.train_dataloaders[0]) // self.config.batch_size * self.config.num_epoch,
        )

        for epoch in range(self.config.num_epoch):
            patience -= 1
            description = "Epoch:{} | Loss:{}".format(epoch, 0)
            iterator = tqdm(self.train_dataloaders[0], desc=description)
            
            # Reset epoch loss accumulator
            epoch_losses = []
            
            for i_batch, sample_batched in enumerate(iterator):
                global_step += 1
                self.model.train()
                self.optimizer.zero_grad()
                inputs = {
                    col: sample_batched[col].to(self.config.device)
                    for col in self.config.inputs_cols
                }

                if self.config.use_amp:
                    with torch.cuda.amp.autocast():
                        outputs = self.model(inputs)
                else:
                    outputs = self.model(inputs)

                targets = sample_batched["polarity"].to(self.config.device)

                if isinstance(outputs, dict) and "loss" in outputs and outputs["loss"] != 0:
                    loss = outputs["loss"]
                else:
                    loss = criterion(outputs["logits"], targets)

                if self.config.auto_device == DeviceTypeOption.ALL_CUDA:
                    loss = loss.mean()

                losses.append(loss.item())
                epoch_losses.append(loss.item()) # Accumulate for epoch average

                # Note: Step-wise WandB logging removed here.

                if self.config.use_amp and self.scaler:
                    self.scaler.scale(loss).backward()
                    self.scaler.step(self.optimizer)
                    self.scaler.update()
                else:
                    loss.backward()
                    self.optimizer.step()

                if self.config.warmup_step >= 0:
                    with self.warmup_scheduler.dampening():
                        self.lr_scheduler.step()

                # Keep step-based evaluation for model saving / patience check (BUT NO WANDB LOGGING)
                if global_step % self.config.log_step == 0:
                    if self.test_dataloader and epoch >= self.config.evaluate_begin:
                        if len(self.valid_dataloaders) > 1:
                            test_acc, f1, _ = self._evaluate_acc_f1(self.valid_dataloaders[0], criterion)
                        else:
                            test_acc, f1, _ = self._evaluate_acc_f1(self.test_dataloader, criterion)
                        
                        self.config.metrics_of_this_checkpoint["acc"] = test_acc
                        self.config.metrics_of_this_checkpoint["f1"] = f1

                        if test_acc > max_fold_acc or f1 > max_fold_f1:
                            if test_acc > max_fold_acc:
                                patience = self.config.patience - 1
                                max_fold_acc = test_acc

                            if f1 > max_fold_f1:
                                max_fold_f1 = f1
                                patience = self.config.patience - 1

                            if self.config.model_path_to_save:
                                if not os.path.exists(self.config.model_path_to_save):
                                    os.makedirs(self.config.model_path_to_save)
                                if save_path:
                                    try:
                                        shutil.rmtree(save_path)
                                    except:
                                        pass
                                save_path = "{0}/{1}_{2}_acc_{3}_f1_{4}/".format(
                                    self.config.model_path_to_save,
                                    self.config.model_name,
                                    self.config.dataset_name,
                                    round(test_acc * 100, 2),
                                    round(f1 * 100, 2),
                                )

                                if test_acc > self.config.max_test_metrics["max_apc_test_acc"]:
                                    self.config.max_test_metrics["max_apc_test_acc"] = test_acc
                                if f1 > self.config.max_test_metrics["max_apc_test_f1"]:
                                    self.config.max_test_metrics["max_apc_test_f1"] = f1

                                save_model(self.config, self.model, self.tokenizer, save_path)

                        postfix = "Dev Acc:{:>.2f}(max:{:>.2f}) Dev F1:{:>.2f}(max:{:>.2f})".format(
                            test_acc * 100,
                            max_fold_acc * 100,
                            f1 * 100,
                            max_fold_f1 * 100,
                        )
                        iterator.set_postfix_str(postfix)
                    elif self.config.save_mode and epoch >= self.config.evaluate_begin:
                        save_model(
                            self.config,
                            self.model,
                            self.tokenizer,
                            save_path + "_{}/".format(loss.item()),
                        )
                
                iterator.set_description("Epoch:{:>3d} | Batch Loss: {:>.4f}".format(epoch, round(loss.item(), 4)))
                iterator.refresh()
            
            # --- End of Epoch Routine: Calculate Epoch Metrics and Log ONCE per epoch ---
            avg_train_loss_epoch = np.mean(epoch_losses) if epoch_losses else 0
            
            # Evaluate on validation set at the end of epoch to get val_loss
            val_loss_epoch = 0
            val_acc_epoch = 0
            val_f1_epoch = 0
            
            # Determine which dataloader to use for epoch validation
            eval_dataloader = None
            if len(self.valid_dataloaders) > 0:
                eval_dataloader = self.valid_dataloaders[0]
            elif self.test_dataloader:
                eval_dataloader = self.test_dataloader
                
            if eval_dataloader:
                 val_acc_epoch, val_f1_epoch, val_loss_epoch = self._evaluate_acc_f1(eval_dataloader, criterion)

            # Get current learning rate for logging
            current_lr = self.optimizer.param_groups[0]['lr']

            wandb.log({
                "epoch/train_loss": avg_train_loss_epoch,
                "epoch/val_loss": val_loss_epoch,
                "epoch/val_acc": val_acc_epoch,
                "epoch/val_f1": val_f1_epoch,
                "epoch/learning_rate": current_lr,
                "epoch": epoch
            })
            self.logger.info(f"End of Epoch {epoch}: Train Loss={avg_train_loss_epoch:.4f}, Val Loss={val_loss_epoch:.4f}")
            # -----------------------------------------------------------

            if patience == 0:
                break

        # Final Evaluation and Logging
        if not self.valid_dataloaders:
            wandb.log({"test/max_acc": max_fold_acc, "test/max_f1": max_fold_f1})
        else:
            fprint("Loading best model: {} and evaluating on test set ".format(save_path))
            self._reload_model_state_dict(save_path)
            max_fold_acc, max_fold_f1, _ = self._evaluate_acc_f1(self.test_dataloader, criterion)
            wandb.log({"test/max_acc": max_fold_acc, "test/max_f1": max_fold_f1})

        # Close WandB run (optional but good practice)
        wandb.finish()

        if self.valid_dataloader or self.config.save_mode:
            del self.train_dataloaders
            del self.test_dataloader
            del self.valid_dataloaders
            del self.model
            cuda.empty_cache()
            time.sleep(3)
            return save_path
        else:
            del self.train_dataloaders
            del self.test_dataloader
            del self.valid_dataloaders
            cuda.empty_cache()
            time.sleep(3)
            return self.model, self.config, self.tokenizer

    def _k_fold_train_and_evaluate(self, criterion):
        # WandB Note: K-Fold with WandB can be tricky.
        # This implementation logs all folds to the same run but you might see jumps in the charts.
        fold_test_acc = []
        fold_test_f1 = []

        save_path_k_fold = ""
        max_fold_acc_k_fold = 0

        losses = []

        self.config.metrics_of_this_checkpoint = {"acc": 0, "f1": 0}
        self.config.max_test_metrics = {"max_apc_test_acc": 0, "max_apc_test_f1": 0}

        for f, (train_dataloader, valid_dataloader) in enumerate(zip(self.train_dataloaders, self.valid_dataloaders)):
            patience = self.config.patience + self.config.evaluate_begin
            if self.config.log_step < 0:
                self.config.log_step = len(self.train_dataloaders[0]) if self.config.log_step < 0 else self.config.log_step

            self.logger.info("***** Running training for Fold {} *****".format(f + 1))
            
            global_step = 0
            max_fold_acc = 0
            max_fold_f1 = 0
            save_path = "{0}/{1}_{2}".format(
                self.config.model_path_to_save,
                self.config.model_name,
                self.config.dataset_name,
            )
            for epoch in range(self.config.num_epoch):
                patience -= 1
                description = "Fold:{} Epoch:{} | Loss:{}".format(f, epoch, 0)
                iterator = tqdm(train_dataloader, desc=description)
                
                epoch_losses = []

                for i_batch, sample_batched in enumerate(iterator):
                    global_step += 1
                    self.model.train()
                    self.optimizer.zero_grad()
                    inputs = {col: sample_batched[col].to(self.config.device) for col in self.config.inputs_cols}

                    if self.config.use_amp:
                        with torch.cuda.amp.autocast():
                            outputs = self.model(inputs)
                    else:
                        outputs = self.model(inputs)

                    targets = sample_batched["polarity"].to(self.config.device)

                    if isinstance(outputs, dict) and "loss" in outputs and outputs["loss"] != 0:
                        loss = outputs["loss"]
                    else:
                        loss = criterion(outputs["logits"], targets)

                    if self.config.auto_device == DeviceTypeOption.ALL_CUDA:
                        loss = loss.mean()

                    losses.append(loss.item())
                    epoch_losses.append(loss.item())
                    
                    # Note: Step-wise WandB logging removed here.

                    if self.config.use_amp and self.scaler:
                        self.scaler.scale(loss).backward()
                        self.scaler.step(self.optimizer)
                        self.scaler.update()
                    else:
                        loss.backward()
                        self.optimizer.step()

                    if self.config.warmup_step >= 0:
                        with self.warmup_scheduler.dampening():
                            self.lr_scheduler.step()

                    if global_step % self.config.log_step == 0:
                        if self.test_dataloader and epoch >= self.config.evaluate_begin:
                            test_acc, f1, _ = self._evaluate_acc_f1(valid_dataloader, criterion)

                            self.config.metrics_of_this_checkpoint["acc"] = test_acc
                            self.config.metrics_of_this_checkpoint["f1"] = f1
                            
                            # Note: Step-wise logging removed here.

                            if test_acc > max_fold_acc or f1 > max_fold_f1:
                                if test_acc > max_fold_acc:
                                    patience = self.config.patience - 1
                                    max_fold_acc = test_acc

                                if f1 > max_fold_f1:
                                    max_fold_f1 = f1
                                    patience = self.config.patience - 1

                                if self.config.model_path_to_save:
                                    if not os.path.exists(self.config.model_path_to_save):
                                        os.makedirs(self.config.model_path_to_save)
                                    if save_path:
                                        try:
                                            shutil.rmtree(save_path)
                                        except:
                                            pass
                                    save_path = "{0}/{1}_{2}_acc_{3}_f1_{4}/".format(
                                        self.config.model_path_to_save,
                                        self.config.model_name,
                                        self.config.dataset_name,
                                        round(test_acc * 100, 2),
                                        round(f1 * 100, 2),
                                    )

                                    if test_acc > self.config.max_test_metrics["max_apc_test_acc"]:
                                        self.config.max_test_metrics["max_apc_test_acc"] = test_acc
                                    if f1 > self.config.max_test_metrics["max_apc_test_f1"]:
                                        self.config.max_test_metrics["max_apc_test_f1"] = f1

                                    save_model(self.config, self.model, self.tokenizer, save_path)

                            postfix = "Dev Acc:{:>.2f}(max:{:>.2f}) Dev F1:{:>.2f}(max:{:>.2f})".format(
                                test_acc * 100,
                                max_fold_acc * 100,
                                f1 * 100,
                                max_fold_f1 * 100,
                            )
                            iterator.set_postfix_str(postfix)

                    iterator.set_description("Fold:{} Epoch:{} | Loss:{}".format(f, epoch, round(loss.item(), 4)))
                    iterator.refresh()
                
                # --- End of Epoch Routine (K-Fold): Calculate Epoch Metrics and Log ONCE per epoch ---
                avg_train_loss_epoch = np.mean(epoch_losses) if epoch_losses else 0
                val_acc_epoch, val_f1_epoch, val_loss_epoch = self._evaluate_acc_f1(valid_dataloader, criterion)

                wandb.log({
                    f"fold_{f}/epoch_train_loss": avg_train_loss_epoch,
                    f"fold_{f}/epoch_val_loss": val_loss_epoch,
                    f"fold_{f}/epoch_val_acc": val_acc_epoch,
                    f"fold_{f}/epoch_val_f1": val_f1_epoch,
                    "epoch": epoch
                })
                # -------------------------------------

                if patience == 0:
                    break
            
            # End of fold
            max_fold_acc, max_fold_f1, _ = self._evaluate_acc_f1(self.test_dataloader, criterion)
            if max_fold_acc > max_fold_acc_k_fold:
                save_path_k_fold = save_path
            fold_test_acc.append(max_fold_acc)
            fold_test_f1.append(max_fold_f1)
            
            self._reload_model_state_dict(save_path_k_fold)

        # End of all folds
        max_test_acc = numpy.max(fold_test_acc)
        max_test_f1 = numpy.max(fold_test_f1)
        
        wandb.log({
            "k_fold/max_test_acc": max_test_acc,
            "k_fold/max_test_f1": max_test_f1
        })
        wandb.finish()

        if os.path.exists("./init_state_dict.bin"):
            os.remove("./init_state_dict.bin")
        
        return save_path

    def _evaluate_acc_f1(self, test_dataloader, criterion):
        """
        Modified to calculate Loss in addition to Acc and F1
        """
        self.model.eval()
        n_test_correct, n_test_total = 0, 0
        t_targets_all, t_outputs_all = None, None
        total_loss = 0
        total_steps = 0
        
        with torch.no_grad():
            for t_batch, t_sample_batched in enumerate(test_dataloader):
                t_inputs = {
                    col: t_sample_batched[col].to(self.config.device)
                    for col in self.config.inputs_cols
                }

                t_targets = t_sample_batched["polarity"].to(self.config.device)
                t_outputs = self.model(t_inputs)

                if isinstance(t_outputs, dict):
                    sen_outputs = t_outputs["logits"]
                    if "loss" in t_outputs and t_outputs["loss"] != 0:
                         batch_loss = t_outputs["loss"]
                    else:
                         batch_loss = criterion(sen_outputs, t_targets)
                else:
                    sen_outputs = t_outputs
                    batch_loss = criterion(sen_outputs, t_targets)

                total_loss += batch_loss.item()
                total_steps += 1

                n_test_correct += (torch.argmax(sen_outputs, -1) == t_targets).sum().item()
                n_test_total += len(sen_outputs)

                if t_targets_all is None:
                    t_targets_all = t_targets
                    t_outputs_all = sen_outputs
                else:
                    t_targets_all = torch.cat((t_targets_all, t_targets), dim=0)
                    t_outputs_all = torch.cat((t_outputs_all, sen_outputs), dim=0)

        test_acc = n_test_correct / n_test_total
        f1 = metrics.f1_score(
            t_targets_all.cpu(),
            torch.argmax(t_outputs_all.cpu(), -1),
            labels=list(range(self.config.output_dim)),
            average=self.config.get("f1_average", "macro"),
        )
        
        avg_loss = total_loss / total_steps if total_steps > 0 else 0
        
        return test_acc, f1, avg_loss

    def _init_misc(self):
        # eta1 and eta2 works only on LSA models
        if "LSA" in str(self.model.models[0].__class__):
            eta_ids = []
            etas = []
            for child in self.model.children():
                if "eta" in str(child.__class__):
                    eta_ids += list(map(id, child.parameters()))
                    etas.append(child)
            base_params = filter(
                lambda p: id(p) not in eta_ids, self.model.models.parameters()
            )
            self.config.eta_lr = (
                self.config.learning_rate * 1000
                if "eta_lr" not in self.config.args
                else self.config.args["eta_lr"]
            )
            self.optimizer = init_optimizer(self.config.optimizer)(
                [
                    {"params": base_params},
                    {
                        "params": etas,
                        "lr": self.config.eta_lr,
                        "weight_decay": self.config.l2reg,
                    },
                ],
                lr=self.config.learning_rate,
                weight_decay=self.config.l2reg,
            )
        else:
            self.optimizer = init_optimizer(self.config.optimizer)(
                self.model.parameters(),
                lr=self.config.learning_rate,
                weight_decay=self.config.l2reg,
            )

    def _cache_or_load_dataset(self):
        pass

    def run(self):
        criterion = nn.CrossEntropyLoss()
        return self._train(criterion)

In [22]:
from typing import Union

from pyabsa.framework.flag_class.flag_template import (
    DeviceTypeOption,
    ModelSaveOption,
    TaskCodeOption,
    TaskNameOption,
)
from pyabsa.framework.trainer_class.trainer_template import Trainer
from pyabsa.tasks.AspectPolarityClassification.configuration.apc_configuration import APCConfigManager
from pyabsa.tasks.AspectPolarityClassification.prediction.sentiment_classifier import SentimentClassifier
from pyabsa.tasks.AspectPolarityClassification.instructor.apc_instructor import APCTrainingInstructor


class APCTrainerWDB(Trainer):
    """Trainer entry point for Aspect Polarity Classification (APC).

    This wrapper connects configuration, datasets and the APC training
    instructor. After initialization, it triggers the standard training
    pipeline and exposes `load_trained_model()` (in the base trainer) to
    obtain a ready-to-use `SentimentClassifier` for inference.
    """

    def __init__(
        self,
        config: APCConfigManager = None,
        dataset=None,
        from_checkpoint: str = None,
        checkpoint_save_mode: int = ModelSaveOption.SAVE_MODEL_STATE_DICT,
        auto_device: Union[bool, str] = DeviceTypeOption.AUTO,
        path_to_save=None,
        load_aug=False,
    ):
        """Initialize the APC training workflow.

        Args:
            config: An `APCConfigManager` instance describing model, tokenizer
                and training hyperparameters.
            dataset: Dataset name, directory path, `DatasetItem`, or list of
                such entries; see docs for supported formats.
            from_checkpoint: Optional checkpoint path to resume training.
            checkpoint_save_mode: One of `ModelSaveOption.*` controlling what
                to save (state_dict, entire model, or fine-tuned PLM).
            auto_device: Device selection strategy or explicit device string.
            path_to_save: Directory to save checkpoints and logs.
            load_aug: Whether to load available augmentation datasets.

        Notes:
            After training, call `load_trained_model()` to obtain an
            inference-ready `SentimentClassifier`.
        """
        super(APCTrainerWDB, self).__init__(
            config=config,
            dataset=dataset,
            from_checkpoint=from_checkpoint,
            checkpoint_save_mode=checkpoint_save_mode,
            auto_device=auto_device,
            path_to_save=path_to_save,
            load_aug=load_aug,
        )

        self.training_instructor = APCTrainingInstructorWandB
        self.inference_model_class = SentimentClassifier
        self.config.task_code = TaskCodeOption.Aspect_Polarity_Classification
        self.config.task_name = TaskNameOption().get(
            TaskCodeOption.Aspect_Polarity_Classification
        )

        self._run()

In [23]:
from pyabsa import ModelSaveOption, DeviceTypeOption


trainer = APCTrainerWDB(
    config=config,
    dataset=dataset,
    from_checkpoint="english",
    # if you want to resume training from our pretrained checkpoints, you can pass the checkpoint name here
    auto_device=DeviceTypeOption.AUTO,
    path_to_save="/kaggle/working/models/",  # set a path to save checkpoints, if it is None, save checkpoints at 'checkpoints' folder
    checkpoint_save_mode=ModelSaveOption.SAVE_MODEL_STATE_DICT,
    load_aug=True,
    # there are some augmentation dataset for integrated datasets, you use them by setting load_aug=True to improve performance
)

[2025-12-21 14:31:00] (2.4.2) Set Model Device: cuda:0
[2025-12-21 14:31:00] (2.4.2) Device Name: Tesla P100-PCIE-16GB


config.json: 0.00B [00:00, ?B/s]

/usr/lib/python3.12/multiprocessing/popen_fork.py:66: DeprecationWarning: This process (pid=24) is multi-threaded, use of fork() may lead to deadlocks in the child.
  self.pid = os.fork()


2025-12-21 14:31:00,733 INFO: PyABSA version: 2.4.2


INFO:fast_lsa_t_v2:PyABSA version: 2.4.2


...

INFO:fast_lsa_t_v2:End of Epoch 4: Train Loss=0.1561, Val Loss=0.4134
Epoch:  5 | Batch Loss: 0.0692:   1%|          | 4/643 [00:00<02:20,  4.54it/s]/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/google/protobuf/internal/well_known_types.py:178: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.FromDatetime(datetime.datetime.utcnow())
Epoch:  5 | Batch Loss: 0.2600:   1%|▏         | 9/643 [00:13<11:24,  1.08s/it, Dev Acc:89.10(max:90.03) Dev F1:89.19(max:90.12)]/usr/local/lib/python3.12/dist-packages/jupyter_client/sessi

2025-12-21 17:16:16,982 INFO: End of Epoch 5: Train Loss=0.1168, Val Loss=0.4252


INFO:fast_lsa_t_v2:End of Epoch 5: Train Loss=0.1168, Val Loss=0.4252


[2025-12-21 17:16:16] (2.4.2) Loading best model: /kaggle/working/models//fast_lsa_t_v2_SEntFiN_acc_90.03_f1_90.12/ and evaluating on test set 


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
wandb: updating run metadata
wandb: uploading output.log; uploading wandb-summary.json
wandb: uploading output.log; uploading wandb-summary.json; uploading config.yaml
wandb: uploading output.log
wandb: uploading history steps 6-6, summary
wandb: 
wandb: Run history:
wandb:               epoch ▁▂▄▅▇█
wandb: epoch/learning_rate ▁▅███▇
wandb:    epoch/train_loss █▄▃▂▁▁
wandb:       epoch/val_acc ▁▇▇▇▇█
wandb:        epoch/val_f1 ▁▇▇▇▇█
wandb:      epoch/val_loss █▁▁▂▇█
wandb:        test/max_acc ▁
wandb:         test/max_f1 ▁
wandb: 
wandb: Run summary:
wandb:               epoch 5
wandb: epoch/learning_rate 1e-05
wandb:    epoch/train_loss 0.11675
wandb:       epoch/val_a